### 원본 데이터 Preprocess

In [ ]:
# 필요한 라이브러리 불러오기
import folium
import pandas as pd
from datetime import timedelta

In [ ]:
# 공유자동차 운행이력 데이터 불러오기
raw_drive = pd.read_csv('/data/공유자동차운행이력.csv')

# 날짜변환
raw_drive['s_date'] = pd.to_datetime(raw_drive['s_date'])
raw_drive['e_date'] = pd.to_datetime(raw_drive['e_date'])

In [ ]:
# 1차 가공 _ 데이터 간격이 2시간 초과인 행이 나타날 때마다 자르기
# 조건1 : 마지막 행인 경우
# 조건2 : 이전 행의 'e_date'와 다음 행의 's_date'의 시간간격이 2시간 초과인 경우
# 조건3 : 'ndevice_id'가 달라진 경우

# start_end : 운행시작 index와 운행종료 index가 튜플로 담긴 리스트
start_end, i = [], 0

for idx in range(len(raw_drive)):
    if idx == len(raw_drive) - 1:
        start_end.append((i, idx))
        break       
    elif raw_drive.at[idx+1, 's_date'] - raw_drive.at[idx, 'e_date'] > timedelta(hours=2):
        start_end.append((i, idx))
        i = idx+1
    elif raw_drive.at[idx, 'ndevice_id'] != raw_drive.at[idx+1, 'ndevice_id']:
        start_end.append((i, idx))
        i = idx+1

# 운행데이터 뭉치의 출발위치와 도착위치를 이은 리스트
first_drive_list = []
for i in start_end:
    first_drive_list.append([raw_drive.at[i[0], 's_lat'], raw_drive.at[i[0], 's_lng'], raw_drive.at[i[1], 'e_lat'], raw_drive.at[i[1], 'e_lng'], raw_drive.at[i[0], 's_gps_odo'], raw_drive.at[i[1], 'e_gps_odo']])
first_drive = pd.DataFrame(data = first_drive_list, columns = ['s_lat', 's_lng', 'e_lat', 'e_lng', 's_gps_odo', 'e_gps_odo'])

In [ ]:
# 2차가공 _ 출발~도착 주행거리가 1km 초과인 행만 가져오기
# e_gps_odo - s_gps_odo가 1000 초과인 것만 추출
first_drive['gps_ds'] = first_drive['e_gps_odo'] - first_drive['s_gps_odo']
second_drive = first_drive[first_drive['gps_ds'] > 1000]

In [ ]:
# 운행시작점 시각화
map_s = folium.Map(location = [36, 128])
for idx, srs in second_drive.iterrows():
  folium.Circle(location = [srs['s_lat'], srs['s_lng']], radius = 20, color = 'red').add_to(map_s)
map_s.save('시작점시각화.html')

In [ ]:
# 운행종료점 시각화
map_e = folium.Map(location = [36, 128])
for idx, srs in second_drive.iterrows():
  folium.Circle(location = [srs['e_lat'], srs['e_lng']], radius = 20, color = 'blue').add_to(map_e)
map_e.save('도착점시각화.html')

In [ ]:
second_drive.to_csv('check1_운행.csv')